In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key



In [2]:
city_data = pd.read_csv("output_data/cities.csv")
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cidreira,-30.1811,-50.2056,65.57,84,2,23.06,BR,1666750038
1,Vila,42.0304,-8.1588,54.48,94,100,7.67,PT,1666750179
2,Šilalė,55.4906,22.1814,51.51,93,90,11.63,LT,1666750179
3,Sūsangerd,31.5608,48.1831,79.12,21,98,6.93,IR,1666750180
4,Dikson,73.5069,80.5464,23.94,76,100,0.27,RU,1666750069


In [4]:
gmaps.configure(api_key = g_key)
locations = city_data[["Lat", "Lng"]].astype(float)
humidity = city_data["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
vacation_spots = city_data.loc[(city_data["Humidity"] < 65)
                                  & (city_data["Cloudiness"] < 15)
                                  & (city_data["Max Temp"] > 75) 
                                  & (city_data["Max Temp"] < 80)]

vacation_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
124,Mahabalipuram,12.6264,80.1722,78.85,62,1,5.19,IN,1666750211
165,Nsanje,-16.9200,35.2620,75.47,63,6,3.58,MW,1666750225
277,Caçu,-18.5567,-51.1308,76.96,36,10,3.36,BR,1666750257
497,Massakory,12.9960,15.7293,79.20,16,6,12.28,TD,1666750397
503,Kiryat Gat,31.6100,34.7642,75.88,31,0,5.03,IL,1666750314
532,Dafeng,23.4517,108.5881,79.66,56,14,6.17,CN,1666750410
558,Porbandar,21.6422,69.6093,77.63,64,1,3.89,IN,1666750417


In [6]:
# hotel_df = pd.DataFrame(vacation_spots)
hotel_df = vacation_spots.reset_index(drop = True)
hotel_df["Location"] = hotel_df["Lat"].astype(str) + "," + hotel_df["Lng"].astype(str)
hotel_df["Hotel Name"] = ""
hotel_df["Address"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Location,Hotel Name,Address
0,Mahabalipuram,12.6264,80.1722,78.85,62,1,5.19,IN,1666750211,"12.6264,80.1722",,
1,Nsanje,-16.9200,35.2620,75.47,63,6,3.58,MW,1666750225,"-16.92,35.262",,
2,Caçu,-18.5567,-51.1308,76.96,36,10,3.36,BR,1666750257,"-18.5567,-51.1308",,
3,Massakory,12.9960,15.7293,79.20,16,6,12.28,TD,1666750397,"12.995999999999999,15.7293",,
4,Kiryat Gat,31.6100,34.7642,75.88,31,0,5.03,IL,1666750314,"31.61,34.7642",,
5,Dafeng,23.4517,108.5881,79.66,56,14,6.17,CN,1666750410,"23.4517,108.5881",,
6,Porbandar,21.6422,69.6093,77.63,64,1,3.89,IN,1666750417,"21.6422,69.6093",,


In [8]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"




location = "23.6139,58.5922"

rankby = "distance"

type = "lodging"

radius = 5000

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
#     params["location"] = f"{lat},{lng}"

    
    params = {"radius": 5000,"type": "lodging", "key": g_key}
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params = params).json()
    
#     
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Address"] = response["results"][0]["vicinity"]
    
    except: 
        print("Missing field... skipping")



In [9]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Location,Hotel Name,Address
0,Mahabalipuram,12.6264,80.1722,78.85,62,1,5.19,IN,1666750211,"12.6264,80.1722",Radisson Blu Resort Temple Bay Mamallapuram,"57 Covelong, East Coast Road, Mahabalipuram"
1,Nsanje,-16.9200,35.2620,75.47,63,6,3.58,MW,1666750225,"-16.92,35.262",Nsanje Discovery Lodge,37G5+H2G
2,Caçu,-18.5567,-51.1308,76.96,36,10,3.36,BR,1666750257,"-18.5567,-51.1308",Império Hotel,"Rua Sena e Melo, 352 - Centro, Caçu"
3,Massakory,12.9960,15.7293,79.20,16,6,12.28,TD,1666750397,"12.995999999999999,15.7293",Chabakari - Am Doukhan,"2Q34+63, Massakory"
4,Kiryat Gat,31.6100,34.7642,75.88,31,0,5.03,IL,1666750314,"31.61,34.7642",OlusHome,"ציבעוני 68, ניר ח""ן"
5,Dafeng,23.4517,108.5881,79.66,56,14,6.17,CN,1666750410,"23.4517,108.5881",Shenglong Hotel,"CJH3+53Q, Zheng Fu Lu, Shang Lin Xian, Nan Nin..."
6,Porbandar,21.6422,69.6093,77.63,64,1,3.89,IN,1666750417,"21.6422,69.6093",Lords Inn Porbandar,"near Circuit House Chaupati, GEB Colony, Porba..."


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]



In [11]:
# coordinates = 

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(locations,info_box_content = hotel_info)

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))